In [ ]:
# 1. Beautiful soup to scrap the data from cnn money site
# 2. Collect fgi & time and update the file in s3

In [107]:
import json
from bs4 import BeautifulSoup as bs
import requests
import datetime
import boto3
import csv

def lambda_handler(event, context):
    bucketname = 'cnn-gfi-bucket'
    bucket_file = 'cnn_fgi.csv'
    local_file = '/tmp/cnn_fgi.csv'
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucketname)
    bucket.download_file(bucket_file, local_file)

    
    url = "https://money.cnn.com/data/fear-and-greed"
    res = requests.get(url)
    # parser = 'lxml'
    soup = bs(res.text, 'html.parser')
    fgi = soup.select("#needleChart > ul > li:nth-of-type(1)")[0].text.split()[4]
    print("fgi", fgi)

    dt = soup.select("#needleAsOfDate")[0].text.split("Last updated")[1].split()
    month = dt[0]
    day = int(dt[1])    
    time = dt[3].split(":")
    hour = int(time[0])
    minute = int(time[1][:2])
    pm = time[1][2:] == "pm"

    hour+=12 if pm else hour

    # convert month string to number (ex: "Jan" -> 1)
    datetime_object = datetime.datetime.strptime(month, "%b")
    
    month_number = datetime_object.month

    x = datetime.datetime(datetime.datetime.now().year, month_number, day, hour, minute)
    date = x.date()
    time = x.time().isoformat(timespec='minutes')
    print('date', date)
    print('time', time)
    
    # write new gfi to local csv file 
    with open(local_file, mode='a') as cnn_fgi:
        writer = csv.writer(cnn_fgi, delimiter=',', quotechar='"')
        writer.writerow([date, time, fgi])
    
    # upload file from tmp to s3 key
    bucket.upload_file(local_file, bucket_file)
    
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }


91
2020-11-25 17:33:00
2020-11-25
17:33
